In [1]:
#########################
######### ROUTINE TO PLOT THE SEASONAL MEAN OF ANY VARIABLE OVER AN ALREADY 
######### SELECTED REGION.
#########################

import cartopy
import cartopy.feature as cfeat
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
import xarray as xr
import numpy as np

import datetime
import pandas

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import ticker
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator
from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.animation as animation
from IPython.display import HTML

In [2]:
#########################
######### DICTIONARIES DEFINITION 
#########################

Range_MLD = {
    'lab' : {
        'vmin' : 20, 
        'vmax' : 2000},
    'gul' : {
        'vmin' : 20, 
        'vmax' : 1000},
    'noe' : {
        'vmin' : 10, 
        'vmax' : 1600},
}


#Colorbar parameters
color_bar = {
    'cmap' : 'YlGnBu',
    'orientation' : 'vertical',
    'title_cbr' : 'depth [m]'
}



reg_ext = {
    'lab': {
        'lon' : (-60, -30),
        'lat' : (50, 65),
        'name' : 'Labrador and Irminger Seas'
    },
    'gul': {
        'lon' : (-75, -45),
        'lat' : (30, 45),
        'name' : 'Gulf Stream'
    },
    'noe': {
        'lon' : (-30, -5),
        'lat' : (45, 60),
        'name' : 'North East Sea'
    }
}

In [3]:
def Grid(data_set):
    x = data_set.longitude
    y = data_set.latitude
    
    x, y = np.meshgrid(x, y)
    return(x, y)


In [4]:
dir_1 = '/home/lgarcia/Documents/data_ARMOR/'

fl_n1 = 'ARMOR_*.nc'

c_armor = xr.open_mfdataset(dir_1 + fl_n1)

mld_a = c_armor.mlotst

In [7]:
import os
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.use('Agg')

N = int(c_armor.time.size)

def generate_one_figure(it=1):
    print(it)
    
    dir_1 = '/home/lgarcia/Documents/data_ARMOR/'
    fl_n1 = 'ARMOR_*.nc'

    c_armor = xr.open_mfdataset(dir_1 + fl_n1)
    
    list_month = [6, 7, 8, 9, 10, 11]
    
    lon1, lon2 = reg_ext['gul']['lon']
    lat1, lat2 = reg_ext['gul']['lat']
    xtik = np.linspace(lon1, lon2, num=5)
    ytik = np.linspace(lat1, lat2, num=4)
    
    
    clim = c_armor.zo.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    clim2 = clim.groupby("time.week").mean("time")
    SSH_A = c_armor.zo.sel(longitude=slice(lon1, lon2), 
                           latitude=slice(lat1, lat2)).groupby("time.week") - clim2
    
    mld_a = c_armor.mlotst.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    #ssh_a = c_armor.zo.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    SSHA = SSH_A.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    
    x_n, y_n = Grid(mld_a)
    time = pandas.to_datetime(mld_a[it].time.values)
    month = time.month
    
    vmin = Range_MLD['gul']['vmin']
    vmax = Range_MLD['gul']['vmax']
    
    projection = ccrs.PlateCarree(360)
    region = [lon1, lon2, lat1, lat2]    #Lon, Lat

    fig, ax = plt.subplots(nrows=1, ncols=1, subplot_kw=dict(projection=projection))           
    ax.set_global()
    ax.coastlines('50m')
    ax.set_xticks(xtik, crs=ccrs.PlateCarree())
    ax.set_yticks(ytik, crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()

    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
    ax.set_extent(region, crs=ccrs.PlateCarree())     #lat, lon
    ax.gridlines()

    if month in list_month:
        contr1 = ax.contourf(x_n, y_n, mld_a[it], levels=np.linspace(15.,500.,20),
                                 cmap='plasma_r', extend='both')
        cbar = fig.colorbar(contr1, ax=ax, shrink=0.55, orientation='vertical',
                     label='MLD [m]', extend='both')  
        cbar.set_clim(10, 500)
    else:
        contr1 = ax.contourf(x_n, y_n, mld_a[it], levels=np.linspace(vmin,vmax,20),
                                 cmap='plasma_r', extend='both')
        cbar = fig.colorbar(contr1, ax=ax, shrink=0.55, orientation='vertical',
                     label='MLD [m]', extend='both')  
        cbar.set_clim(10, 3500)
        
    contr2 = ax.contour(x_n, y_n, SSHA[it,0], 15, linewidths=0.5, colors='k')

    ax.set_title(time)
    fig.savefig(os.path.abspath(os.path.sep.join(["../dummy_images","GulSS_%0.4d.png" % it])))
    plt.close(fig)
    return None

                       
print('Use %i processes' % multiprocessing.cpu_count() ) 

with multiprocessing.Pool() as pool:
    pool.map(generate_one_figure, np.arange(0,730))     
    
##mencoder "mf://dummy_images/Gul_*.png" -mf fps=10 -o Gulf.avi -ovc lavc -lavcopts vcodec=msmpeg4v2:vbitrate=2500
#ffmpeg -r 3 -f image2 -s 1920x1080 -i dummy_images/GulA_%04d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p -q:v 1 Gul-A.mp4

Use 4 processes


/usr/lib/python3/dist-packages/matplotlib/__init__.py:1352: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


0
46
92
138


/usr/local/lib/python3.5/dist-packages/numpy/ma/core.py:6449: MaskedArrayFutureWarning: In the future the default for ma.maximum.reduce will be axis=0, not the current None, to match np.maximum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)
/usr/local/lib/python3.5/dist-packages/numpy/ma/core.py:6449: MaskedArrayFutureWarning: In the future the default for ma.minimum.reduce will be axis=0, not the current None, to match np.minimum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)
/usr/local/lib/python3.5/dist-packages/numpy/ma/core.py:6449: MaskedArrayFutureWarning: In the future the default for ma.maximum.reduce will be axis=0, not the current None, to match np.maximum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)
/usr/local/lib/python3.5/dist-packages/numpy/ma/core.py:6449: MaskedArrayFutureWarning: In the future the default for ma.minimum.reduce will be axis=0, not the current N

184
230


/usr/local/lib/python3.5/dist-packages/numpy/ma/core.py:6449: MaskedArrayFutureWarning: In the future the default for ma.maximum.reduce will be axis=0, not the current None, to match np.maximum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)
/usr/local/lib/python3.5/dist-packages/numpy/ma/core.py:6449: MaskedArrayFutureWarning: In the future the default for ma.minimum.reduce will be axis=0, not the current None, to match np.minimum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)
/usr/local/lib/python3.5/dist-packages/numpy/ma/core.py:6449: MaskedArrayFutureWarning: In the future the default for ma.maximum.reduce will be axis=0, not the current None, to match np.maximum.reduce. Explicitly pass 0 or None to silence this warning.
  return self.reduce(a)
/usr/local/lib/python3.5/dist-packages/numpy/ma/core.py:6449: MaskedArrayFutureWarning: In the future the default for ma.minimum.reduce will be axis=0, not the current N

276
322


MemoryError: 

## Labrador + Irminger Seas

In [ ]:
import os
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.use('Agg')

r = 'lab'
N = int(c_armor.time.size)

def generate_one_figure(it=1):
    print(it)
    
    dir_1 = '/home/lgarcia/Documents/data_ARMOR/'
    fl_n1 = 'ARMOR_*.nc'

    c_armor = xr.open_mfdataset(dir_1 + fl_n1)
    
    lon1, lon2 = reg_ext[r]['lon']
    lat1, lat2 = reg_ext[r]['lat']
    xtik = np.linspace(lon1, lon2, num=5)
    ytik = np.linspace(lat1, lat2, num=4)
    
    mld_a = c_armor.mlotst.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    ssh_a = c_armor.zo.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    x_n, y_n = Grid(mld_a)
    time = pandas.to_datetime(mld_a.time.values[it])
    
    vmin = Range_MLD[r]['vmin']
    vmax = Range_MLD[r]['vmax']
    
    projection = ccrs.PlateCarree(360)
    region = [lon1, lon2, lat1, lat2]    #Lon, Lat

    fig, ax = plt.subplots(nrows=1, ncols=1, subplot_kw=dict(projection=projection))           
    ax.set_global()
    ax.coastlines('50m')
    ax.set_xticks(xtik, crs=ccrs.PlateCarree())
    ax.set_yticks(ytik, crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()

    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
    ax.set_extent(region, crs=ccrs.PlateCarree())     #lat, lon
    ax.gridlines()
    
    if month in list_month:
        contr1 = ax.contourf(x_n, y_n, mld_a[it], levels=np.linspace(15.,500.,20),
                                 cmap='plasma_r', extend='both')
        cbar = fig.colorbar(contr1, ax=ax, shrink=0.55, orientation='vertical',
                     label='MLD [m]', extend='both')  
        cbar.set_clim(10, 500)
    else:
        contr1 = ax.contourf(x_n, y_n, mld_a[it], levels=np.linspace(vmin,vmax,20),
                                 cmap='plasma_r', extend='both')
        cbar = fig.colorbar(contr1, ax=ax, shrink=0.55, orientation='vertical',
                     label='MLD [m]', extend='both')  
        cbar.set_clim(10, 3500)
        
    contr2 = ax.contour(x_n, y_n, SSHA[it,0], 15, linewidths=0.5, colors='k')
    
    ax.set_title(time)
    fig.savefig(os.path.abspath(os.path.sep.join([".","dummy_images","Lab_%0.4d.png" % it])))
    plt.close(fig)
    return None



print('Use %i processes' % multiprocessing.cpu_count() ) 

with multiprocessing.Pool() as pool:
    pool.map(generate_one_figure, np.arange(0,N))     
    
##mencoder "mf://dummy_images/*.png" -mf fps=10 -o Lab.mp4 -ovc lavc -lavcopts vcodec=msmpeg4v2:vbitrate=1500

## North East

In [ ]:
import os
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.use('Agg')

r = 'noe'
N = int(c_armor.time.size)

def generate_one_figure(it=1):
    print(it)
    
    dir_1 = '/home/lgarcia/Documents/data_ARMOR/'
    fl_n1 = 'ARMOR_*.nc'

    c_armor = xr.open_mfdataset(dir_1 + fl_n1)
    
    lon1, lon2 = reg_ext[r]['lon']
    lat1, lat2 = reg_ext[r]['lat']
    xtik = np.linspace(lon1, lon2, num=5)
    ytik = np.linspace(lat1, lat2, num=4)
    
    mld_a = c_armor.mlotst.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    ssh_a = c_armor.zo.sel(longitude=slice(lon1, lon2), latitude=slice(lat1, lat2))
    x_n, y_n = Grid(mld_a)
    time = pandas.to_datetime(mld_a.time.values[it])
    
    vmin = Range_MLD[r]['vmin']
    vmax = Range_MLD[r]['vmax']
    
    projection = ccrs.PlateCarree(360)
    region = [lon1, lon2, lat1, lat2]    #Lon, Lat

    fig, ax = plt.subplots(nrows=1, ncols=1, subplot_kw=dict(projection=projection))           
    ax.set_global()
    ax.coastlines('50m')
    ax.set_xticks(xtik, crs=ccrs.PlateCarree())
    ax.set_yticks(ytik, crs=ccrs.PlateCarree())
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()

    ax.xaxis.set_major_formatter(lon_formatter)
    ax.yaxis.set_major_formatter(lat_formatter)
    ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
    ax.set_extent(region, crs=ccrs.PlateCarree())     #lat, lon
    ax.gridlines()
    
    contr1 = ax.contourf(x_n, y_n, mld_a[it], levels=np.linspace(vmin,vmax,20),
                                 cmap='plasma_r', extend='both')
    contr2 = ax.contour(x_n, y_n, ssh_a[it,0], 15, linewidths=0.5, colors='k')
    cbar = fig.colorbar(contr1, ax=ax, shrink=0.55, orientation='vertical',
                 label='MLD [m]', extend='both')  
    cbar.set_clim(20,3500)
    
    ax.set_title(time)
    fig.savefig(os.path.abspath(os.path.sep.join([".","dummy_images","Noe_%0.4d.png" % it])))
    plt.close(fig)
    return None



print('Use %i processes' % multiprocessing.cpu_count() ) 

with multiprocessing.Pool() as pool:
    pool.map(generate_one_figure, np.arange(0,N))     
    
##mencoder "mf://dummy_images/Noe*.png" -mf fps=10 -o North.mp4 -ovc lavc -lavcopts vcodec=msmpeg4v2:vbitrate=1500
#ffmpeg -r 3 -f image2 -s 1920x1080 -i dummy_images/Noe_%04d.png -vcodec libx264 -crf 25  -pix_fmt yuv420p -q:v 1 Noe.mp4
